In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from showit import image
from scipy.stats import binom
import matplotlib.pyplot as plt
import geopandas
import rasterio
from rasterio import Affine
from tqdm import tqdm

In [ ]:
def integrated_risk(p):
    return (1 - binom.cdf(0, 20, p)) * 100

In [ ]:
ds = xr.open_zarr(
    "https://carbonplan.blob.core.windows.net/carbonplan-forests/risks/results/web/fire.zarr"
).load()

In [ ]:
counties = geopandas.read_file("cb_2021_us_county_500k/cb_2021_us_county_500k.shp")

In [ ]:
exclude = list(["02", "15", "60", "66", "69", "72", "78"])
counties = counties[list(map(lambda x: x not in exclude, counties["STATEFP"]))]

In [ ]:
counties = counties.to_crs("EPSG:5070").reset_index()

In [ ]:
get_mask = lambda geometry: rasterio.features.geometry_mask(
    [geometry],
    out_shape=ds.lat.shape,
    transform=Affine(4000, 0.0, -2493045.0, 0.0, -4000, 3310005.0),
    all_touched=False,
    invert=True,
)

In [ ]:
scenarios = ["ssp245", "ssp370", "ssp585"]
columns = ["state_code", "state_name", "county_code", "county_name"] + list(ds.year.values)
data = {key: [] for key in scenarios}
dfs = {key: [] for key in scenarios}

for index, county in tqdm(counties.iterrows()):
    mask = np.tile(get_mask(county["geometry"]), [12, 1, 1])
    means = ds.where(mask).mean(["x", "y"])
    for scenario in scenarios:
        values = integrated_risk(means[scenario].values)
        data[scenario].append(
            [
                county["STATEFP"],
                county["STUSPS"],
                county["COUNTYFP"],
                county["NAME"],
            ]
            + list(values)
        )

for scenario in scenarios:
    dfs[scenario] = pd.DataFrame(data[scenario], columns=columns)

In [ ]:
dfs["ssp245"].dropna().reset_index(drop=True).to_csv(
    "CarbonPlan-Fire-Risk-SSP-4.5.csv", index=False, header=True
)
dfs["ssp370"].dropna().reset_index(drop=True).to_csv(
    "CarbonPlan-Fire-Risk-SSP-7.0.csv", index=False, header=True
)
dfs["ssp585"].dropna().reset_index(drop=True).to_csv(
    "CarbonPlan-Fire-Risk-SSP-8.5.csv", index=False, header=True
)

In [ ]:
len(dfs["ssp585"])

In [ ]:
counties["fire"] = dfs["ssp585"]["2050"]

In [ ]:
counties.plot(column="fire", cmap="hot", vmin=0, vmax=50, figsize=(15, 10), legend=True)

In [ ]:
counties.plot(column="fire", cmap="hot", figsize=(15, 10))

In [ ]:
counties.plot(column="fire", vmin=0, vmax=25)